<div align="center">

# YOLOv5 simple object detector using a class-based approach

</div>

The YOLOv5 is a powerful object detection algorithm that combines high accuracy and real-time detection speeds. This Colab notebook demonstrates how to implement a simple object detector using a class-based approach, allowing us to detect objects in both static images and videos.


## Importing libraries, module and files

In [1]:
!git clone https://github.com/mohamedamine99/YOLOv5-object-detection

Cloning into 'YOLOv5-object-detection'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 223 (delta 10), reused 38 (delta 5), pack-reused 174
Receiving objects: 100% (223/223), 435.02 MiB | 15.34 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Updating files: 100% (92/92), done.


In [2]:
import os
from pathlib import Path
import shutil
import time
import random

from collections import Counter


import cv2
import matplotlib.pyplot as plt
import numpy as np


# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

In [3]:
%pip install ultralytics  # install
import ultralytics
ultralytics.checks()  # checks

Ultralytics YOLOv8.0.54 🚀 Python-3.9.16 torch-1.13.1+cu116 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/107.7 GB disk)


### Setting up paths and working dirs 

In [53]:
class Yolov5_ObjectDetector:
    def __init__(self, model_name, labels, colors,
                 conf=0.25, iou=0.45,
                 classes=None, max_det=1000):
        # constructor code here
        self.model = torch.hub.load('ultralytics/yolov5', model_name, force_reload=True, verbose = False)
        self.colors = colors
        self.model_name = model_name
        self.labels = labels
        self.model.conf = conf
        self.model.iou = iou
        self.model.classes = classes
        self.model.max_det = max_det

    
    def print_results(self, results):

        labels_list = [self.labels[int(r[5])] for r in results]
        counter_dict = Counter(labels_list)
        #print(results)
        print(f"model : {self.model_name}")
        print("Detected objects : ", end = '')
        for key in counter_dict.keys():
            print(f"{key} : {counter_dict[key]} " , end = ' ')
        
        print()
        #print(counter_dict)


    def run_img_detection(self, img : np.ndarray, verbose = True):
        if img.ndim != 3 or img.shape[2]!=3 :
            raise ValueError("input img should be a 3-dimensional numpy array with 3 colors")

        results = self.model(img)
        results = np.array(results.xyxy[0])
        if verbose :
            print("----------------------------")
        
            print(f"image shape : {img.shape}")
            self.print_results(results)

        return(results)

    
    def run_img_detection_from_path(self, img_path, verbose = True):
        if not os.path.exists(img_path):
            raise ValueError(f"File path {img_path} does not exist.")
        
        img = cv2.imread(img_path)
        img_name = os.path.basename(img_path)

        if verbose : 
            print("\n----------------------------")
            print(f"{img_name} :")

        results = self.run_img_detection(img, verbose)        
        return (results)


    def print_detections_on_image(self, detections: np.ndarray, img: np.ndarray):
        if img.ndim != 3 :
            raise ValueError("input img should be a 3-dimensional numpy array")

        bbx_thickness = (img.shape[0] + img.shape[1]) // 500

        for r in detections:
            # Extract object class and confidence score
            score = r[4] * 100
            r = r.astype(int)
            class_id = r[5]

            # Calculate font scale based on object size
            fontScale = (((r[2] - r[0]) / img.shape[0]) + ((r[3] - r[1]) / img.shape[1])) / 2 * 1.5

            # Draw bounding box, a centroid and label on the image
            im = cv2.putText(img,f"{self.labels[class_id]} {score:,.2f}%" , 
                            (r[0],r[1] - 5), cv2.FONT_HERSHEY_COMPLEX, 
                        fontScale,  self.colors[class_id], 1, cv2.LINE_AA)
            
            im = cv2.rectangle(im, (r[0],r[1]), (r[2],r[3]), self.colors[class_id], bbx_thickness)

            center_coordinates = ((r[2] + r[0])//2, (r[3] + r[1]) // 2)
            im =  cv2.circle(im, center_coordinates, 2 , (0,0,255), -1)

        return im



    def save_img_detection(self, img, save_dir, file_name ):

        if not os.path.exists(save_dir):
            raise ValueError(f"File path {save_dir} does not exist.")

        # save resulting images in their corresponding folders
        save_file = os.path.join(save_dir, self.model_name, file_name)
        print(f"Saving Detection Results of {file_name} to {save_file}")
        cv2.imwrite(save_file ,img)
        
        

        
    
    def run_multiple_imgs_detection_from_path(self, images_path, save_dir):
    
        for img_name in os.listdir(images_path):
            img_path = os.path.join(images_path, img_name)
            img = cv2.imread(img_path)
            results = self.run_img_detection_from_path(img_path)
            img = self.print_detections_on_image(results, img)
            self.save_img_detection(img, save_dir, img_name)

    def run_video_detection_from_path(self, video_path, save_dir, output_FPS = 15, output_format = '.avi', verbose = False):
        if not os.path.exists(video_path):
            raise ValueError(f"File path {video_path} does not exist.")
        
       # Open input video file
        cap = cv2.VideoCapture(video_path)
        print("\n----------------------------")

        # Get video name 
        vid_name = os.path.basename(video_path)
        print(vid_name, end = ' : ')

        # Get frame dimensions and print information about input video file
        width  = int(cap.get(3) )  # get `width` 
        height = int(cap.get(4) )  # get `height` 
        print((width,height))
        print(video_path)
        print(self.model_name)

        # Set bounding box thickness based on video dimensions
        bbx_thickness = (height + width) // 500

        # Define output video file
        save_file = os.path.join(save_dir, vid_name[:-4] + output_format)
        print('saving to :' + save_file)

        # define an output VideoWriter  object
        out = cv2.VideoWriter(save_file,
                            cv2.VideoWriter_fourcc(*"MJPG"),
                            output_FPS,(width,height))

        # Check if the video is opened correctly
        if not cap.isOpened():
            print("Error opening video stream or file")

        # Read the video frames
        while cap.isOpened():
            ret, frame = cap.read()

            # If the frame was not read successfully, break the loop
            if not ret:
                print("Error reading frame")
                break

            # Run object detection on the frame and calculate FPS
            beg = time.time()
            results = self.run_img_detection(frame, verbose= False)
            fps = 1 / (time.time() - beg)

            # Display FPS on frame
            frame = cv2.putText(frame,f"FPS : {fps:,.2f}" , 
                                (5,15), cv2.FONT_HERSHEY_COMPLEX, 
                            0.5,  (0,0,255), 1, cv2.LINE_AA)
            
            frame = self.print_detections_on_image(results, frame)

            # append frame to the video file
            out.write(frame)
            
            # the 'q' button is set as the
            # quitting button you may use any
            # desired button of your choice

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # After the loop release the cap 
        cap.release()
        out.release()

        





In [10]:
# Set up paths and working directories
coco_names_file = '/content/YOLOv5-object-detection/coco.names'  # Path to file containing COCO object class names

results_path = '/content/results'  # Path to directory where result images will be saved
video_results_path = '/content/video_results'  # Path to directory where result videos will be saved

test_imgs_path = '/content/YOLOv5-object-detection/test imgs'  # Path to directory containing test images
test_vids_path = '/content/YOLOv5-object-detection/test vids'  # Path to directory containing test videos


### loading COCO class names 

In [11]:
# Reading the COCO dataset 80 class names from the coco names file
labels = []
with open(coco_names_file, 'rt') as coco_file:
    labels = coco_file.read().rstrip('\n').rsplit('\n')
    
print(labels)

# generate random color for each label
colors = []
for _ in labels:
    rand_tuple = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    colors.append(rand_tuple)


['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


### Loading YOLOv5 variants

In [47]:
detector = Yolov5_ObjectDetector('yolov5n', labels,colors)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-19 Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


In [48]:
results = detector.run_img_detection_from_path('/content/YOLOv5-object-detection/test imgs/people crossing the street.jpg')
img = cv2.imread('/content/YOLOv5-object-detection/test imgs/2 dogs.PNG')
#detector = detector.run_multiple_imgs_detection_from_path(images_path, save_dir)


----------------------------
people crossing the street.jpg :
----------------------------
image shape : (976, 976, 3)
model : yolov5n
Detected objects : person : 5  car : 8  truck : 1  


In [49]:
#os.makedirs('content/results__2')
detector.run_multiple_imgs_detection_from_path('/content/YOLOv5-object-detection/test imgs', 'content/results__2')


----------------------------
highway.PNG :
----------------------------
image shape : (303, 491, 3)
model : yolov5n
Detected objects : car : 9  truck : 3  bus : 1  person : 1  
Saving Detection Results of highway.PNG to content/results__2/yolov5n/highway.PNG

----------------------------
nyc street.PNG :
----------------------------
image shape : (333, 479, 3)
model : yolov5n
Detected objects : person : 14  car : 3  truck : 1  
Saving Detection Results of nyc street.PNG to content/results__2/yolov5n/nyc street.PNG

----------------------------
2 cats.PNG :
----------------------------
image shape : (293, 514, 3)
model : yolov5n
Detected objects : cat : 2  
Saving Detection Results of 2 cats.PNG to content/results__2/yolov5n/2 cats.PNG

----------------------------
street 2.PNG :
----------------------------
image shape : (259, 502, 3)
model : yolov5n
Detected objects : bus : 1  person : 13  car : 2  bicycle : 4  
Saving Detection Results of street 2.PNG to content/results__2/yolov5n/s

In [50]:
# Define a list of YOLOv5 variant model names to be loaded
yolov5_variants_names = ['yolov5n', 'yolov5s', 'yolov5m', 'yolov5l','yolov5x', 
                   'yolov5n6','yolov5s6', 'yolov5m6', 'yolov5l6','yolov5x6' ]


detectors = []
# Loop through the list of model names to load each model and 
# creating Directories for Results and Video Results for YOLOv5 Variants

for yolo_name in yolov5_variants_names:
    detector = Yolov5_ObjectDetector(yolo_name, labels,colors)
    detectors.append(detector)

    new_dir = os.path.join(results_path, yolo_name)
    if os.path.isdir(new_dir):
        print(f"{new_dir} already exists")

    else:
        os.makedirs(new_dir)

    new_dir = os.path.join(video_results_path, yolo_name)
    if os.path.isdir(new_dir):
        print(f"{new_dir} already exists")

    else:
        os.makedirs(new_dir)


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-19 Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


/content/results/yolov5n already exists
/content/video_results/yolov5n already exists


requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-19 Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


/content/results/yolov5s already exists
/content/video_results/yolov5s already exists


requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-19 Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


...

/content/results/yolov5m6 already exists
/content/video_results/yolov5m6 already exists


requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-19 Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5l6 summary: 476 layers, 76726332 parameters, 0 gradients
Adding AutoShape... 


/content/results/yolov5l6 already exists
/content/video_results/yolov5l6 already exists


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-19 Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 


/content/results/yolov5x6 already exists
/content/video_results/yolov5x6 already exists


## Object Detection on Images in a Directory Using YOLOv5 Variants




In [51]:
for detector in detectors:
    detector.run_multiple_imgs_detection_from_path(test_imgs_path, results_path)


----------------------------
highway.PNG :
----------------------------
image shape : (303, 491, 3)
model : yolov5n
Detected objects : car : 9  truck : 3  bus : 1  person : 1  
Saving Detection Results of highway.PNG to /content/results/yolov5n/highway.PNG

----------------------------
nyc street.PNG :
----------------------------
image shape : (333, 479, 3)
model : yolov5n
Detected objects : person : 14  car : 3  truck : 1  
Saving Detection Results of nyc street.PNG to /content/results/yolov5n/nyc street.PNG

----------------------------
2 cats.PNG :
----------------------------
image shape : (293, 514, 3)
model : yolov5n
Detected objects : cat : 2  
Saving Detection Results of 2 cats.PNG to /content/results/yolov5n/2 cats.PNG

----------------------------
street 2.PNG :
----------------------------
image shape : (259, 502, 3)
model : yolov5n
Detected objects : bus : 1  person : 13  car : 2  bicycle : 4  
Saving Detection Results of street 2.PNG to /content/results/yolov5n/street 2.

## Object Detection on Videos in a Directory Using YOLOv5 Variants


In [54]:
detector = Yolov5_ObjectDetector('yolov5n', labels,colors)
detector.run_video_detection_from_path('/content/YOLOv5-object-detection/test vids/traffic.mp4', 
                                       '/content', output_FPS = 15, output_format = '.avi')

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-19 Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 



----------------------------
traffic.mp4 : (640, 360)
/content/YOLOv5-object-detection/test vids/traffic.mp4
yolov5n
saving to :/content/traffic.avi
Error reading frame


## Saving results

In [25]:
!zip -r results.zip /content/results

  adding: content/results/ (stored 0%)
  adding: content/results/yolov5s6/ (stored 0%)
  adding: content/results/yolov5s6/highway.PNG (deflated 0%)
  adding: content/results/yolov5s6/nyc street.PNG (deflated 0%)
  adding: content/results/yolov5s6/2 cats.PNG (deflated 1%)
  adding: content/results/yolov5s6/street 2.PNG (deflated 1%)
  adding: content/results/yolov5s6/2 dogs.PNG (deflated 0%)
  adding: content/results/yolov5s6/people crossing the street.jpg (deflated 0%)
  adding: content/results/yolov5x6/ (stored 0%)
  adding: content/results/yolov5x6/highway.PNG (deflated 0%)
  adding: content/results/yolov5x6/nyc street.PNG (deflated 0%)
  adding: content/results/yolov5x6/2 cats.PNG (deflated 1%)
  adding: content/results/yolov5x6/street 2.PNG (deflated 1%)
  adding: content/results/yolov5x6/2 dogs.PNG (deflated 0%)
  adding: content/results/yolov5x6/people crossing the street.jpg (deflated 0%)
  adding: content/results/yolov5l6/ (stored 0%)
  adding: content/results/yolov5l6/highway.

In [ ]:
!zip -r video_results.zip /content/video_results